In [1]:
! pip install streamlit
!pip install langchain-google-genai
!pip install langchain langchain-google-genai google-generativeai
! pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.9 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 

INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Using cached langchain_google_genai-2.1.0-py3-none-any.whl.metadata (3.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.5 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.16
    Uninstalling google-ai-generativelanguage-0.6.16:
      Successfully uninstalled

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [8]:
%%writefile aitutor1.py
import streamlit as st
import uuid
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_message_histories import SQLChatMessageHistory

# Load API Key
with open("api.txt") as f:
    API_KEY = f.read().strip()

# Initialize Chat Model
chat_model = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-1.5-pro")

# Define Chat Prompt Template
chat_template = ChatPromptTemplate.from_messages([
    SystemMessage(content="""You are an expert and intelligent Data Science tutor.
        Engage in a helpful and interactive learning session for the users and clear their doubts with examples and give code if necessary.
        Your answer should be crisp and easily understandable for the users. Maintain context based on previous discussions."""),
    MessagesPlaceholder(variable_name="chat_history"),  # Stores previous chat
    HumanMessagePromptTemplate.from_template("{human_input}")  # Latest query
])

# Initialize Output Parser
chat_parser = StrOutputParser()

# Function to get session history from SQLite
def get_session_message_history_from_db(session_id):
    return SQLChatMessageHistory(session_id=session_id, connection="sqlite:///chat_history.db")

# Create Conversation Chain
chain = chat_template | chat_model | chat_parser
conversation_chain = RunnableWithMessageHistory(
    chain,
    get_session_message_history_from_db,
    input_messages_key="human_input",
    history_messages_key="chat_history"
)

# Streamlit App Layout
st.set_page_config(page_title="AI Data Science Tutor", page_icon="🤖", layout="centered")

# Username Input
if "username" not in st.session_state:
    with st.form("user_form"):
        username = st.text_input("Enter your name:", key="username_input")
        submit = st.form_submit_button("Start Chat")
        if submit and username.strip():
            st.session_state.username = username.strip()
            st.session_state.session_id = f"{username}_{uuid.uuid4().hex}"
            st.session_state.chat_history = []
            st.rerun()

else:
    session_id = st.session_state.session_id
    st.title(f"🤖 AI Data Science Tutor for {st.session_state.username}")

    # Get chat history from the database
    chat_history = get_session_message_history_from_db(session_id)

    # ✅ FIX: Ensure session history updates with the database messages
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = chat_history.messages.copy()

    # Sidebar - Chat History
    with st.sidebar:
        st.header("Chat History")
        if st.session_state.chat_history:
            first_message = next(
                (msg.content[:50] + "..." for msg in st.session_state.chat_history if isinstance(msg, HumanMessage)),
                "No chat history yet."
            )
            st.text(first_message)

            # Convert chat history to text format
            chat_text = "\n\n".join([f"{type(msg).__name__}: {msg.content}" for msg in st.session_state.chat_history])
            chat_txt = chat_text.encode('utf-8')
            st.download_button("Download", chat_txt, "chat_history.txt", "text/plain")
        else:
            st.write("No chat history yet.")

    # Chat Interface
    st.subheader("Ask Your Questions")
    chat_container = st.container()

    # ✅ FIX: Display messages correctly in the chat UI
    with chat_container:
        for msg in st.session_state.chat_history:
            role = "You" if isinstance(msg, HumanMessage) else "AI Tutor"
            with st.chat_message("user" if isinstance(msg, HumanMessage) else "assistant"):
                st.write(f"**{role}:** {msg.content}")

    # User Input
    human_input = st.chat_input("Ask me a question:")

    if human_input:
        with st.chat_message("user"):
            st.write(f"**You:** {human_input}")

        # ✅ FIX: Send only the latest user message but keep chat history in context
        response = conversation_chain.invoke(
            {"human_input": human_input, "chat_history": st.session_state.chat_history},
            config={"configurable": {"session_id": session_id}}
        )

        # ✅ FIX: Append only new messages (avoids repetition)
        chat_history.add_user_message(human_input)
        chat_history.add_ai_message(response)
        st.session_state.chat_history.append(HumanMessage(content=human_input))
        st.session_state.chat_history.append(AIMessage(content=response))

        with st.chat_message("assistant"):
            st.write(f"**AI Tutor:** {response}")

Overwriting aitutor1.py


In [9]:
!npm install localtunnel
!streamlit run /content/aitutor1.py &>/content/logs.txt &
!npx localtunnel --port 8501 & curl ipv4.icanhazip.com

⠙⠹⠸⠼⠴⠦⠧⠇
up to date, audited 23 packages in 1s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠇35.196.248.183
⠙your url is: https://nice-jobs-run.loca.lt
